<a href="https://colab.research.google.com/github/j2damax/principles-of-ds-cw02/blob/main/online_retail_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import library and load the dataset
import pandas as pd

In [ ]:
# read csv file
df = pd.read_csv("online_retail.csv")
df.head()

In [ ]:
#Check no.of rows and colunms
df.shape

In [ ]:
#Check the data types
df.dtypes

In [ ]:
#check missing values
df.isna().sum()

In [ ]:
#Drop the customerID with missing values
df = df.dropna(subset=['CustomerID'])

In [ ]:
#Fill missing description with unknown
df['Description'] = df['Description'].fillna('Unknown')

In [ ]:
#Ensure no missing values are available in the dataset
print(df.isnull().sum())

In [ ]:
#Check duplicated rows
df.duplicated().sum()

In [ ]:
# Remove negative values in quantity and prices colunm since those values cannot be negative
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

In [ ]:
# Convert InvoiceNo and StockCode to string
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)

# Convert InvoiceDate to datetime format if we need to do time based analysis
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Convert CustomerID to string
df['CustomerID'] = df['CustomerID'].astype(str)

# Convert Country to categorical type
df['Country'] = df['Country'].astype('category')

df.dtypes

# Feature Engineering for analysis

In [ ]:
# 01Create TotalPrice to get the revenue per transaction
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

In [ ]:
#02Create additional columns by categorizing the Date for the analysis of seasonal trends

# categorize date components into year, month, day, weekday,working hour and weekend
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.weekday  # Monday = 0, Sunday = 6
df['Hour'] = df['InvoiceDate'].dt.hour

# Create a feature for working hours
df['IsWorkingHour'] = df['Hour'].apply(lambda x: 1 if 9 <= x <= 18 else 0)

# Check if the purchase was made on a weekend
df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

#Display created columns
print(df[['Year', 'Month', 'Day', 'Weekday', 'Hour', 'IsWeekend']].head())

In [ ]:
#03Customer recency
import datetime
latest_date = df['InvoiceDate'].max()  # Get last transaction date
df['Recency'] = df.groupby('CustomerID')['InvoiceDate'].transform(lambda x: (latest_date - x.max()).days)

#print 5 rows of customer recency by ensuring one row per customerID
print(df[['CustomerID', 'InvoiceDate', 'Recency']].drop_duplicates(subset=['CustomerID']).head(5))

#print a random customer's recency
print(df[['CustomerID', 'Recency']].drop_duplicates().sample(5))


In [ ]:
#04Create customer-based aggregations to find the total spend, total number of purchases and average order value per customer


# Total spend per customer
customer_spending = df.groupby('CustomerID')['TotalPrice'].sum().reset_index()

# Total number of purchases per customer
customer_frequency = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()

# Average order value per customer
customer_avg_order = df.groupby('CustomerID')['TotalPrice'].mean().reset_index()

print(customer_spending.head())
print(customer_frequency.head())
print(customer_avg_order.head())

In [ ]:
#05Find the popular product based on the purchase quantity grouping by description

# Aggregate total quantity sold per product (grouping by Description)
popular_products = df.groupby('Description')['Quantity'].sum().reset_index()

# Sort by most sold products
popular_products = popular_products.sort_values(by='Quantity', ascending=False)

# Add a flag for popular products (Top 100)
df['IsPopularProduct'] = df['Description'].apply(lambda x: 1 if x in popular_products['Description'][:100].values else 0)

# Display top products
print(popular_products.head(10))

# Exploratory Data Analysis

In [ ]:
#get summary statistics of dataset
df.describe()

In [ ]:
#Monthly sales analysis

import matplotlib.pyplot as plt
import seaborn as sns

monthly_sales = df.groupby(['Year', 'Month'])['TotalPrice'].sum().reset_index()

plt.figure(figsize=(8,4))
sns.lineplot(data=monthly_sales, x='Month', y='TotalPrice', hue='Year', marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregate total quantity sold per product
popular_products = df.groupby('Description')['Quantity'].sum().reset_index()

# Sort by highest quantity and take the top 10
top_10_products = popular_products.sort_values(by='Quantity', ascending=False).head(10)

# Plot bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x='Quantity', y='Description', data=top_10_products, palette='viridis')

# Add labels and title
plt.xlabel('Total Quantity Sold')
plt.ylabel('Product Description')
plt.title('Top 10 Popular Products by Quantity')
plt.show()
